### Пример проекта по визуализации геоданных

Библиотеки, которые нам понадобились:

**pandas** для сбора и анализа данных
**geocoder** - для получения коорднат из названия стран
**folium** - для нанесения данных на карту

Установка:
в пустой ячейке введите:
!pip install geocoder folium


https://proglib.io/p/python-web-map/
пошаговая инструкция по фолиуму

In [37]:
!pip install geocoder folium tqdm

In [117]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd

req = requests.get('https://www.worldometers.info/coronavirus/')
soup = BeautifulSoup(req.text)

In [118]:
table = soup.find('table')
df = pd.read_html(table.prettify(), header =0)[0]
df.rename(columns=lambda x: x.replace('  ', ' '), inplace=True)


In [ ]:
import geocoder
g = geocoder.arcgis('Russia')
g.json

In [ ]:
import geocoder
from tqdm import tqdm



def get_cors(country_name:str) -> tuple:
    g = geocoder.arcgis(country_name)
    return g.json['lat'], g.json['lng']

##Раскомментировать, если нужно получить словарь в первый раз 
# con_cors = {}

# for country in tqdm(df['Country, Other']):
#     con_cors[country] = get_cors(country)
    
# con_cors

In [119]:
cors = requests.get('https://raw.githubusercontent.com/alnagaev/python_lessons/master/cors.json').json()



In [120]:
lats = []
lngs = []
for k, v in cors.items():
    lats.append(v[0])
    lngs.append(v[1])


        
df['lat'] = lats
df['lng'] = lngs

In [122]:
df = df[df['#'].notna()]

In [123]:
df

,#,"Country, Other",Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,Active Cases,"Serious, Critical",Tot Cases/ 1M pop,Deaths/ 1M pop,Total Tests,Tests/ 1M pop,Population,lat,lng
1,1.0,USA,1644640,"+23,743",97635.0,"+1,281",397295.0,1149710.0,17100.0,4972.0,295.0,13882648.0,41968.0,3.307905e+08,39.398703,-99.414619
2,2.0,Brazil,330890,"+19,969",21048.0,+966,135430.0,174412.0,8318.0,1558.0,99.0,735224.0,3462.0,2.123933e+08,-10.769946,-53.073467
3,3.0,Russia,326448,"+8,894",3249.0,+150,99825.0,223374.0,2300.0,2237.0,22.0,8126626.0,55689.0,1.459278e+08,61.989526,96.805372
4,4.0,Spain,281904,"+1,787",28628.0,+688,196958.0,56318.0,1152.0,6030.0,612.0,3037840.0,64977.0,4.675285e+07,40.227950,-3.646063
5,5.0,UK,254195,"+3,287",36393.0,+351,NaN,NaN,1559.0,3747.0,536.0,3231921.0,47635.0,6.784716e+07,53.408387,-1.969560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,211.0,Western Sahara,6,NaN,NaN,NaN,6.0,0.0,NaN,10.0,NaN,NaN,NaN,5.955830e+05,24.658503,-13.139486
212,212.0,Anguilla,3,NaN,NaN,NaN,3.0,0.0,NaN,200.0,NaN,NaN,NaN,1.498800e+04,18.224603,-63.059623
213,213.0,Lesotho,2,+1,NaN,NaN,NaN,2.0,NaN,0.9,NaN,NaN,NaN,2.140374e+06,-29.582325,28.246694
214,214.0,Saint Pierre Miquelon,1,NaN,NaN,NaN,1.0,0.0,NaN,173.0,NaN,NaN,NaN,5.797000e+03,46.856133,-56.322830


In [124]:
df['ratio'] = round(df['Total Deaths']/df['Total Cases']*100, 2)
df[['Country, Other', 'ratio', ]].dropna().head()

 

/home/politlen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,"Country, Other",ratio
1,USA,5.94
2,Brazil,6.36
3,Russia,1.00
4,Spain,10.16
5,UK,14.32


In [129]:
from math import ceil
def get_radius(percent):
    if percent > 10:
        return 10
    elif percent < 1:
        return 1
    else:
        return ceil(percent)

In [126]:
def color_producer(el):
    if el < 10:
        return 'green'
    elif 10 <= el < 100:
        return 'yellow'
    else:
        return 'red'

In [127]:
dropped = df[['Country, Other', 'ratio', 'lat', 'lng', 'Total Cases', 'Deaths/ 1M pop']].dropna()
dropped['percent'] = round(dropped['Total Cases']/dropped['Total Cases'].sum(), 2)*100
dropped.head()

,"Country, Other",ratio,lat,lng,Total Cases,Deaths/ 1M pop,percent
1,USA,5.94,39.398703,-99.414619,1644640,295.0,31.0
2,Brazil,6.36,-10.769946,-53.073467,330890,99.0,6.0
3,Russia,1.00,61.989526,96.805372,326448,22.0,6.0
4,Spain,10.16,40.227950,-3.646063,281904,612.0,5.0
5,UK,14.32,53.408387,-1.969560,254195,536.0,5.0


In [132]:
import folium
fol_map = folium.Map(location=[36.567348, 103.930027], zoom_start=2)
for lat, lon, percent, total, country, ratio, dr  in zip(
                            dropped['lat'].tolist(), 
                            dropped['lng'].tolist(), 
                            dropped['percent'].tolist(),
                            dropped['Total Cases'].tolist(),
                            dropped['Country, Other'].tolist(),
                            dropped['ratio'].tolist(),
                            dropped['Deaths/ 1M pop'].tolist()
                                 ):
    folium.CircleMarker(location=[lat, lon], radius = get_radius(percent), popup= str(country)+' '
                        + str(total)+" total cases\n" + str(ratio) + " ratio ",
                            fill=True, 
                            fill_color=color_producer(dr), color = color_producer(dr)).add_to(fol_map)

In [133]:
fol_map